# Imports

In [1]:
'''Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

import csv
import keras
import numpy as np
from keras import regularizers
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from itertools import chain
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def limit_mem():
    keras.backend.get_session().close()
    cfg = keras.backend.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    keras.backend.set_session(keras.backend.tf.Session(config=cfg))

get_available_gpus()

['/device:GPU:0']

In [3]:
def show_weights_distribution(xrange=0.4):
    #NO BIAS WEIGHTS
    all_weights = []
    for x in model.layers:
        if(x.get_weights()) :
            all_weights.append(np.reshape(x.get_weights()[0], -1))
    all_weights = np.concatenate(all_weights)
    plt.hist(all_weights, bins=80, range=(-xrange, xrange))

def get_threshold():
    #NO BIAS WEIGHTS
    all_weights = []
    for x in model.layers:
        if(x.get_weights()) :
            all_weights.append(np.reshape(x.get_weights()[0], -1))
    all_weights = np.abs(np.concatenate(all_weights))
    all_weights = np.sort(all_weights)
    threshold_value = all_weights[int(zero_part * len(all_weights)) + 1]
    return threshold_value
    
def get_masks(threshold_value):
    zero_masks = []
    for x in model.layers:
        if(x.get_weights()) :
            zero_masks.append(np.abs(x.get_weights()[0]) <= threshold_value)
            
    return zero_masks

def set_null_weights_by_mask(zero_mask):
    mask_iter = 0
    for x in model.layers:
        if(x.get_weights()) :
            old_weights, old_bias = x.get_weights()
            old_weights[zero_mask[mask_iter]] = 0
            mask_iter += 1
            x.set_weights([old_weights, old_bias])

# Model achitecture

In [6]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name_prefix = 'keras_cf10_'
l1_coef = 0.0001 # * 10, * 100, * ??? was 0.00001
zero_part = 0.0
zero_step = 0.05
segment_runs = 5
min_los_delta = 0.001

# 01 version - stock model, dropouts, no L1/L2
# 02 version - stock model, no dropouts, no L1/L2
# 03 version - stock model, no dropouts, L1 0.01 kernel_regularizer

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))#, kernel_regularizer=regularizers.l1(l1_coef)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))#, kernel_regularizer=regularizers.l1(l1_coef)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))#, kernel_regularizer=regularizers.l1(l1_coef)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))#, kernel_regularizer=regularizers.l1(l1_coef)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))#, kernel_regularizer=regularizers.l1(l1_coef)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))#, kernel_regularizer=regularizers.l1(l1_coef)))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
# maybe adam, /5 or /10 default parametrs

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, "empty_model")
model.save(model_path)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
cb_lists = [
    EarlyStopping(monitor='val_loss', 
                  min_delta=0.1 * min_los_delta, 
                  patience=10, 
                  verbose=0, 
                  mode='auto', 
                  baseline=None, 
                  restore_best_weights=False)
]

In [8]:
def example_loop():
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                  batch_size=batch_size,
                  epochs=epochs,
                  validation_data=(x_test, y_test),
                  shuffle=True)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            zca_epsilon=1e-06,  # epsilon for ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            # randomly shift images horizontally (fraction of total width)
            width_shift_range=0.1,
            # randomly shift images vertically (fraction of total height)
            height_shift_range=0.1,
            shear_range=0.,  # set range for random shear
            zoom_range=0.,  # set range for random zoom
            channel_shift_range=0.,  # set range for random channel shifts
            # set mode for filling points outside the input boundaries
            fill_mode='nearest',
            cval=0.,  # value used for fill_mode = "constant"
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,  # randomly flip images
            # set rescaling factor (applied before any other transformation)
            rescale=None,
            # set function that will be applied on each input
            preprocessing_function=None,
            # image data format, either "channels_first" or "channels_last"
            data_format=None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split=0.0)
    
        # Compute quantities required for feature-wise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        datagen.fit(x_train)
    
        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            workers=4,
                            steps_per_epoch=len(x_train)/batch_size,
                            callbacks=cb_lists)

In [9]:
def custom_loop():
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    
    enable_early_cut_off = False
    enable_zeroing_weights = True
    enable_saving_best_model = True
    zero_mode = False
    zero_mask = []
    prev_loss = 0
    best_loss = 999999999999
    best_acc = 0
    # save best modes
    
    for i in range(0, epochs):
        gen = datagen.flow(x_train, y_train, batch_size=batch_size)
        
        for j in range(0, int(len(x_train)/batch_size)):
            x_tr, y_tr = next(gen)
            model.train_on_batch(x_tr, y_tr)
            # zero here!!!!
        
        print('Finish epoch: ' + str(i))
        epoch_loss, epoch_acc = model.evaluate(x_test, y_test)
        print('Validation loss: ' + str(epoch_loss) + ', acc ' + str(epoch_acc))
        
        if (enable_early_cut_off):
            if (abs(epoch_loss - prev_loss) < min_los_delta):
                print('Exit because loss stoped changing. epoch_loss ' 
                      + str(epoch_loss) 
                      + ' prev_loss' 
                      + str(prev_loss))
                break
            prev_loss = epoch_loss
        
        if (enable_zeroing_weights):
            if (i%segment_runs==0):
                if not zero_mode:
                    #print('Start zeroing mode')
                    zero_mask = get_masks(get_threshold())
                #print('Invert zeroing mode')
                zero_mode = not zero_mode
            if (zero_mode):
                set_null_weights_by_mask(zero_mask)
                #print('Use zero mask')
                
        if(enable_saving_best_model):
            if(epoch_loss < best_loss):
                best_loss = epoch_loss
                best_acc = epoch_acc
                
                if not os.path.isdir(save_dir):
                    os.makedirs(save_dir)
                model_path = os.path.join(save_dir, model_name)
                model.save(model_path)
                print('Saved best model at %s ' % model_path)
    return best_loss, best_acc
            

# Active cell

In [ ]:
while(zero_part < 1.0):
    model = keras.models.load_model('saved_models/empty_model')
    model_name = model_name_prefix + str(zero_part) + 'zrpt'
    loss_result, acc_result = custom_loop()
    print('After runs, ' + str(loss_result) + ' loss and ' + str(acc_result) + ' acc')

    # show_weights_distribution()

    with open('results.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([str(zero_part)] + [str(loss_result)] + [str(acc_result)])
    zero_part += zero_step
        
#limit_mem()

# Load best model
#model = keras.models.load_model('saved_models/keras_cifar10_trained_03.h5')
#scores = model.evaluate(x_test, y_test, verbose=1)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])

In [ ]:
#!jupyter notebook stop 11440
os._exit(0)

In [ ]:
#Load model and show it's score
model = keras.models.load_model('saved_models/keras_cifar10_trained_02.h5')

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
with open('results.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in spamreader:
        print(', '.join(row))

In [ ]:
model = keras.models.load_model('saved_models/keras_cf10_0.9500000000000003zrpt')

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

show_weights_distribution(1)

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
!pip uninstall GraphViz

In [ ]:
model_name = "example_arch"
loss_result, acc_result = custom_loop()
print('After runs, ' + str(loss_result) + ' loss and ' + str(acc_result) + ' acc')

Finish epoch: 0
10000/10000 [==============================] - 1s 54us/step
Validation loss: 1.5546368354797364, acc 0.4399
Saved best model at /home/nikitam/saved_models/example_arch 
Finish epoch: 1
10000/10000 [==============================] - 1s 51us/step
Validation loss: 1.397330514907837, acc 0.4924
Saved best model at /home/nikitam/saved_models/example_arch 
Finish epoch: 2
10000/10000 [==============================] - 1s 54us/step
Validation loss: 1.3088701847076416, acc 0.5252
Saved best model at /home/nikitam/saved_models/example_arch 
Finish epoch: 3
10000/10000 [==============================] - 1s 57us/step
Validation loss: 1.2009690359115601, acc 0.5719
Saved best model at /home/nikitam/saved_models/example_arch 
Finish epoch: 4
10000/10000 [==============================] - 0s 47us/step
Validation loss: 1.1674182235717774, acc 0.5811
Saved best model at /home/nikitam/saved_models/example_arch 
Finish epoch: 5
10000/10000 [==============================] - 1s 59us/step
